In [6]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

#read fdaya
data=pd.read_csv("https://raw.githubusercontent.com/salammemphis/ML/main/dataset/amazon_cells_labelled.txt",sep="\t")
text=data.iloc[:,0]
label=data.iloc[:,1]

x_train, x_test,y_train,y_test=train_test_split(text,label,test_size=0.3,random_state=42)

#tokenize and count
vectorizer=CountVectorizer(stop_words="english")
x_train=vectorizer.fit_transform(x_train)
#print(vectorizer.get_feature_names())
print(x_train.shape, text.shape)
print(x_train.toarray())



(699, 1318) (999,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:

multnomial=MultinomialNB()
multnomial.fit(x_train,y_train)

logisticRegression=LogisticRegression()
logisticRegression.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
test_cnts=vectorizer.transform(x_test)
y_pred=multnomial.predict(test_cnts)
print("model's accuracy Bayes",accuracy_score(y_pred,y_test))

y_pred=logisticRegression.predict(test_cnts)
print("model's accuracy Logistic",accuracy_score(y_pred,y_test))

model's accuracy Bayes 0.7866666666666666
model's accuracy Logistic 0.7766666666666666


In [11]:
email=["bad head phone. broke after two days","I enjoyed the features"]
test_cnts=vectorizer.transform(email)
print(test_cnts.shape)
y_pred=multnomial.predict(test_cnts)
print(y_pred)

(2, 1318)
[0 1]


In [2]:
import pandas as pd
import re
data=pd.read_csv("https://raw.githubusercontent.com/salammemphis/ML/main/dataset/amazon_cells_labelled.txt",sep="\t")

data.columns=["text","label"]
data.head()


,text,label
0,"Good case, Excellent value.",1
1,Great for the jawbone.,1
2,Tied to charger for conversations lasting more...,0
3,The mic is great.,1
4,I have to jiggle the plug to get it to line up...,0


In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data["text"]=data['text']


In [48]:
max_fatures = 2000
maxlen=0
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
maxlen=len(X[0])
print(data['text'][0:5])
print(X[0:5])

Y=data["label"]
Y = to_categorical(Y)#one hot encoding

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

0                            good case excellent value
1                                great for the jawbone
2    tied to charger for conversations lasting more...
3                                     the mic is great
4    i have to jiggle the plug to get it to line up...
Name: text, dtype: object
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  21  48  52 263]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  16  11   1 405]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0 789   8  83  11 406 539  84  50 790 791 139]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   1 330   5  16]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   2  22   8 792   1
  161   8  68   4   8 407  58 140   8  68 331 141]]
(799, 30) (799, 2)
(200, 30) (200, 2)


In [43]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))#max_fatures=vocab size, length of embedding vector,
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 30, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)


Epoch 1/7
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
25/25 - 3s - loss: 0.6888 - accuracy: 0.5307
Epoch 2/7
25/25 - 3s - loss: 0.6101 - accuracy: 0.7384
Epoch 3/7
25/25 - 3s - loss: 0.4013 - accuracy: 0.8473
Epoch 4/7
25/25 - 3s - loss: 0.2229 - accuracy: 0.9212
Epoch 5/7
25/25 - 3s - loss: 0.1117 - accuracy: 0.9675
Epoch 6/7
25/25 - 3s - loss: 0.0796 - accuracy: 0.9825
Epoch 7/7
25/25 - 3s - loss: 0.0510 - accuracy: 

In [49]:
email=["bad head phone. broke after two days","I enjoyed the features"]
X = tokenizer.texts_to_sequences(email)
X = pad_sequences(X,maxlen=maxlen)
print(X)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 121   9 195  56 117 173]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   2   1 421]]


In [58]:
np.argmax(model.predict(X), axis=0)

array([0, 1])